Pandas and duckdb

###  Task 0

In [ ]:
# Task 0

import duckdb
from pathlib import Path

duckdb_path = "data/sakila.duckdb"
Path(duckdb_path).unlink(missing_ok=True)

with duckdb.connect(duckdb_path) as conn, open("sql/load_sakila.sql") as ingest_script:
    conn.sql(ingest_script.read())

    description = conn.sql("DESC;").df()
    films = conn.sql("FROM film;").df()

        

films.head()  

In [69]:
description.head()

,database,schema,name,column_names,column_types,temporary
0,sakila,main,actor,"[actor_id, first_name, last_name, last_update]","[DOUBLE, VARCHAR, VARCHAR, TIMESTAMP]",False
1,sakila,main,address,"[address_id, address, address2, district, city...","[BIGINT, VARCHAR, VARCHAR, VARCHAR, BIGINT, VA...",False
2,sakila,main,category,"[category_id, name, last_update]","[BIGINT, VARCHAR, TIMESTAMP]",False
3,sakila,main,city,"[city_id, city, country_id, last_update]","[BIGINT, VARCHAR, BIGINT, TIMESTAMP]",False
4,sakila,main,country,"[country_id, country, last_update]","[BIGINT, VARCHAR, TIMESTAMP]",False


### Task 1

###  Read all data into dictionary of pandas dataframes

In [ ]:
dfs = {}

with duckdb.connect(duckdb_path) as conn:
    for name in description["name"]:
        dfs[name] = conn.sql(f"FROM {name};").df()

dfs.keys()

In [ ]:
# task 1a)

length_of_movie=duckdb.sql("""    
    SELECT title, length AS movie_length
    FROM films
    WHERE length > 180;         
    """).df()

length_of_movie.head()

In [ ]:
# task 1b)

love_word=duckdb.sql("""
SELECT title, rating, length, description
FROM films
WHERE title ILIKE '%love%';
""").df()

love_word.head()

In [ ]:
# task 1c)

length_statistik=duckdb.sql("""
SELECT
MIN(length) AS shortest,
MAX(length) AS longest,
AVG(length) AS average,
MEDIAN(length) AS median 
FROM films;                          
""").df()

length_statistik.head()



In [ ]:
# task 1d)

expensive_movies=duckdb.sql("""
SELECT title, rental_rate, rental_duration, (rental_rate / rental_duration) AS cost_per_day
FROM films
ORDER BY cost_per_day DESC
LIMIT 10;                              
""").df()

expensive_movies

In [ ]:
# task 1e)

top_actors=duckdb.sql("""
SELECT a.actor_id, a.first_name, a.last_name, COUNT(*) AS film_count
FROM actor a
JOIN film_actor fa ON a.actor_id = fa.actor_id
GROUP BY a.actor_id, a.first_name, a.last_name
ORDER BY film_count DESC
LIMIT 10;
                      
""").df()

top_actors

CatalogException: Catalog Error: Table with name actor does not exist!
Did you mean "pg_attrdef"?